In [34]:
import h2o
# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,8 hours 36 mins
H2O_cluster_timezone:,America/Asuncion
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,1 month and 20 days
H2O_cluster_name:,H2O_from_python_matiaslopez_nsue9n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,13.71 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [35]:
import pandas as pd
y = PARAMETERS['TARGET_COLUMN']
balanced_df = pd.read_csv(PARAMETERS['BALANCED_DATASET_CSV_PATH'], index_col=False)
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [36]:
PARAMETERS = {
	'BALANCED_DATASET_CSV_PATH': '../balanced_output_145.csv',
	'TARGET_COLUMN': 'Protestas',
	'USE_RANDOM_UNDERSAMPLING': True,
	'NO_COMPLAINTS_%': 0.50,
	'SAVE_LEADER_MODEL': True,
	'SAVED_MODEL_PATH': 'finalModel',
	'TEST_DATA_CSV': '../test_2023.csv',
	
}
import h2o
from h2o.automl import H2OAutoML
aml = H2OAutoML(max_models=42, seed=23, stopping_metric="AUCPR", sort_metric="AUCPR")

## Random undersampling

In [37]:
balanced_df_true = balanced_df[balanced_df[y] == True]
balanced_df_false = balanced_df[balanced_df[y] == False]
n_cmp = len(balanced_df_true)
n_no_cmp = len(balanced_df_false)
n_cmp / (n_cmp + n_no_cmp)

0.12663430420711974

In [38]:
# Random undersampling
if PARAMETERS['USE_RANDOM_UNDERSAMPLING']:
	balanced_df_true = balanced_df[balanced_df[y] == True]
	balanced_df_false = balanced_df[balanced_df[y] == False]
	n_cmp = len(balanced_df_true)
	n_no_cmp = len(balanced_df_false)
	total = n_cmp + n_no_cmp
	sample_false = balanced_df_false.sample(int(total*PARAMETERS['NO_COMPLAINTS_%'])) #Sample 70% no complaints 30 % complaints
	#TODO: Analisis de sensibilidad de este parametro 
	balanced_df = pd.concat([balanced_df_true, sample_false])

## Training

In [39]:
train = h2o.H2OFrame.from_python(balanced_df)
# Identify predictors and response
x = train.columns
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [40]:
# Run AutoML for 10 base models
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leader
# board
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |
18:17:30.783: _train param, Dropping bad and constant columns: [tender.procurementMethod_open, parties.details.legalEntityTypeDetail buyer_15, parties.details.legalEntityTypeDetail buyer_14, parties.details.legalEntityTypeDetail buyer_13, parties.details.legalEntityTypeDetail buyer_12, parties.details.legalEntityTypeDetail buyer_11, parties.details.legalEntityTypeDetail buyer_10, parties.details.EntityType payee_1, parties.details.EntityType payee_2, parties.details.EntityType payee_3, parties.details.EntityType payee_4, planning.items.classification.id.n1_1_57, planning.items.classification.id.n1_1_58, Monto faltante, parties.details.legalEntityTypeDetail notifiedSupplier_18, tender.status_active, tender.statusDetails_En Convocatoria (Abierta), parties.details.EntityType candidate_2, parties.details.EntityType candidate_1, parties.details.EntityType candidate_4, parties.details.legalEntityTypeDetail candidate_1, awards.statusDetails_2, parties.details.EntityType can

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_5_20230809_181730,0.905502,0.955504,0.183482,0.125451,0.226647,0.0513687
StackedEnsemble_BestOfFamily_1_AutoML_5_20230809_181730,0.902951,0.954191,0.186121,0.127631,0.228134,0.0520451
GBM_grid_1_AutoML_5_20230809_181730_model_7,0.898576,0.951605,0.193581,0.127944,0.232018,0.0538325
GBM_grid_1_AutoML_5_20230809_181730_model_8,0.89837,0.951308,0.19287,0.127559,0.231799,0.0537307
XGBoost_grid_1_AutoML_5_20230809_181730_model_5,0.895416,0.949574,0.206437,0.129165,0.233897,0.0547077
GBM_grid_1_AutoML_5_20230809_181730_model_5,0.891273,0.949536,0.198362,0.129288,0.236825,0.0560859
XGBoost_grid_1_AutoML_5_20230809_181730_model_7,0.890263,0.946309,0.214707,0.134751,0.237722,0.0565119
XGBoost_grid_1_AutoML_5_20230809_181730_model_8,0.886024,0.946839,0.204265,0.13264,0.238962,0.0571029
XGBoost_grid_1_AutoML_5_20230809_181730_model_3,0.885575,0.946357,0.20567,0.131676,0.239548,0.0573833
XGBoost_grid_1_AutoML_5_20230809_181730_model_10,0.882962,0.946019,0.206852,0.133671,0.240932,0.0580482


In [41]:
if PARAMETERS['SAVE_LEADER_MODEL']:
	model_path = h2o.save_model(model=aml.leader, path=PARAMETERS['SAVED_MODEL_PATH'], force=True)
	print(model_path)

/Users/matiaslopez/Documents/tesis/tesis-model/Training/finalModel/StackedEnsemble_AllModels_1_AutoML_5_20230809_181730


## Testing 2023 Data

In [42]:
df_test = pd.read_csv(PARAMETERS['TEST_DATA_CSV'], index_col=False)
test_frame = h2o.H2OFrame.from_python(df_test)
test_frame[y] = test_frame[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [43]:
def get_performance(model, test: h2o.H2OFrame):
	cm = model.model_performance(test).confusion_matrix().to_list()
	perf_t = cm[1][1] / (cm[1][0] + cm[1][1])
	perf_f = cm[0][0] / (cm[0][1] + cm[0][0])
	return perf_t, perf_f

In [44]:
best = ''
best_score_t = 0
best_score_f = 0
df_leaderboard = aml.leaderboard.as_data_frame()
model_ids = df_leaderboard['model_id']
for model_id in model_ids:
	model = h2o.get_model(model_id)
	score_t, score_f = get_performance(model, test_frame)
	print(model_id, score_t, score_f)
	if score_t > best_score_t and score_f > best_score_f:
		best_score_t = score_t
		best_score_f = score_f
		best = model_id

print(best, best_score_t, best_score_f)

StackedEnsemble_AllModels_1_AutoML_5_20230809_181730 0.7528089887640449 0.9012750915288473
StackedEnsemble_BestOfFamily_1_AutoML_5_20230809_181730 0.7071629213483146 0.9241257416992804
GBM_grid_1_AutoML_5_20230809_181730_model_7 0.7514044943820225 0.8890291629844717
GBM_grid_1_AutoML_5_20230809_181730_model_8 0.7636938202247191 0.8867567226360308
XGBoost_grid_1_AutoML_5_20230809_181730_model_5 0.7075140449438202 0.915793460421664
GBM_grid_1_AutoML_5_20230809_181730_model_5 0.7587780898876404 0.8925640701931574
XGBoost_grid_1_AutoML_5_20230809_181730_model_7 0.7092696629213483 0.9000126246686024
XGBoost_grid_1_AutoML_5_20230809_181730_model_8 0.735252808988764 0.8948365105415983
XGBoost_grid_1_AutoML_5_20230809_181730_model_3 0.7124297752808989 0.9189496275722763
XGBoost_grid_1_AutoML_5_20230809_181730_model_10 0.7257724719101124 0.9144047468753945
XGBoost_grid_1_AutoML_5_20230809_181730_model_1 0.7061095505617978 0.9087236460042923
GBM_grid_1_AutoML_5_20230809_181730_model_4 0.74403089

In [48]:
best_model = h2o.get_model('GBM_1_AutoML_5_20230809_181730')
best_model.model_performance(test_frame).confusion_matrix()

,False,True,Error,Rate
False,7019.0,902.0,0.1139,(902.0/7921.0)
True,619.0,2229.0,0.2173,(619.0/2848.0)
Total,7638.0,3131.0,0.1412,(1521.0/10769.0)


In [49]:
model_path = h2o.save_model(model=best_model, path="theLastModel", force=True)

In [50]:
best_model

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_5_20230809_181730


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    104                104                         216732                 15           15           15            32            313           159.625

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.03816244404081575
RMSE: 0.1953521027294453
LogLoss: 0.14311960783806138
Mean Per-Class Error: 0.08325318393505343
AUC: 0.9850900716987869
AUCPR: 0.9571976581511548
Gini: 0.9701801433975739

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3927527732955818
       False    True    Error    Rate
-----  -------  ------  -------  ----------------
False  45396    954     0.0206   (954.0/46350.0)
True   1713     10026   0.1459   (1713.0/11739.0)
Total  47109    10980   0.0459   (2667.0/58089.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.392753     0.882609  187
max f2                       0.190714     0.898763  256
max f0point5                 0.549388     0.921782  140
max accuracy                 0.424934     0.954243  177
max precision                0.998027     1         0
max recall                   0.0168093    1         380
max specificity              0.998027     1         0
max absolute_mcc             0.392753     0.854849  187
max min_per_class_accuracy   0.213747     0.933161  247
max mean_per_class_accuracy  0.216446     0.933322  246
max tns                      0.998027     46350     0
max fns                      0.998027     11695     0
max fps                      0.00238067   46350     399
max tps                      0.0168093    11739     380
max tnr                      0.998027     1         0
max fnr                      0.998027     0.996252  0
max fpr                      0.00238067   1         399
max tpr                      0.0168093    1         380

Gains/Lift Table: Avg response rate: 20.21 %, avg score: 20.22 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100019                   0.990399           4.94838     4.94838            1                0.993929    1                           0.993929            0.0494931       0.0494931                  394.838   394.838            0.0494931
2        0.0200038                   0.984667           4.94838     4.94838            1                0.987768    1                           0.990848            0.0494931       0.0989863                  394.838   394.838            0.0989863
3        0.0301262                   0.977152           4.94838     4.94838            1                0.981069    1                           0.987563            0.0500894       0.149076                   394.838   394.838            0.149076
4        0.0400248                   0.968535           4.94838     4.94838            1                0.973129    1                           0.983993            0.048982        0.198058                   394.838   394.838            0.198058
5        0.0500783                   0.960027           4.94838     4.94838            1        